In [0]:
# Standard library imports
import json
import time
import subprocess
# Third-party imports
import requests

# Databricks SDK imports
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import catalog, jobs, pipelines

# Install pymssql silently
subprocess.run(
    ["pip", "install", "pymssql"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# Import pymssql and its error class
import pymssql
from pymssql import OperationalError

In [0]:
def switch_pipeline(param_api_token, param_api_url, param_pipeline_id, param_action="stop", param_full_refresh=False, param_cause="Triggered by API"):
    try:
        if param_action == "start":
            v_endpoint = f"{param_api_url}/{param_pipeline_id}/updates"
            v_headers_dict = {
                "Authorization": f"Bearer {param_api_token}",
                "Content-Type": "application/json"
            }
            v_payload_dict = {
                "full_refresh": param_full_refresh,
                "cause": param_cause
            }
        elif param_action == "stop":
            v_endpoint = f"{param_api_url}/{param_pipeline_id}/stop"
            v_headers_dict = {
                "Authorization": f"Bearer {param_api_token}",
                "Content-Type": "application/json"
            }
            v_payload_dict = {}
        else:
            raise ValueError("Invalid action. Must be 'start' or 'stop'.")
    except Exception as v_error:
        raise Exception("Error:", v_error)

    v_response = requests.post(v_endpoint, headers=v_headers_dict, data=json.dumps(v_payload_dict))
    print(f"{param_action.capitalize()} response status:", v_response.status_code)
    try:
        print("Response body:", v_response.json())
    except Exception as v_error:
        print("No JSON response:", v_response.text)

In [0]:
def get_pipeline_status(param_api_token, param_api_url, param_pipeline_id, param_state):
    # The get_pipeline_status function retrieves the current status of a pipeline, such as IDLE, RUNNING, or STOPPING.
    try:
        v_headers_dict = {
            "Authorization": f"Bearer {param_api_token}",
            "Content-Type": "application/json"
        }

        v_status_url = f"{param_api_url}/{param_pipeline_id}"
        v_response = requests.get(v_status_url, headers=v_headers_dict)

        if v_response.status_code == 200:
            v_data_dict = v_response.json()

            if param_state == 'start':
                return v_data_dict.get("state") #v_data_dict.get("latest_updates")[0].get("state")
            elif param_state == "end":
                return v_data_dict.get("latest_updates")[0].get("state")
            else:
                raise ValueError("Invalid value for state. Must be 'start' or 'end'.")
        elif v_response.status_code == 404:
            return "NOT_FOUND"
        else:
            print("Failed to retrieve pipeline status. Status code:", v_response.status_code)
            return None
    except Exception as v_error:
        raise Exception("Error:", v_error)


In [0]:
# #Temporary Solution while we are figuring out how to extract the data in system attributes

def get_server_config(param_environment = None):
    if param_environment.lower() == "dev":
        return {
            "username": "AnalyticsOwner",
            "password": dbutils.secrets.get(scope="AnalyticsDevKv01Scope", key="AnalyticsOwner"),
            "hostname": f"nprod-analytics-dev-01-sql.database.windows.net",
            "port":1433}
        
    elif param_environment.lower() == "qa":
        return {
            "username": "AnalyticsOwner",
            "password": dbutils.secrets.get(scope="nprod-analytics-qae-kvm", key="AnalyticsOwner"),
            "hostname": f"nprod-analytics-qae-01-sql.database.windows.net",
            "port":1433}
    
    elif param_environment.lower() == "stg":
        return {
            "username": "AnalyticsOwner",
            "password": dbutils.secrets.get(scope="prod-analytics-stg-kvm", key="AnalyticsOwner"),
            "hostname": f"prod-analytics-stg-01-sql.database.windows.net",
            "port":1433}
    
    elif param_environment.lower() == "prod":
        return {
            "username": "AnalyticsOwner",
            "password": dbutils.secrets.get(scope="prod-analytics-kvm", key="AnalyticsOwner"),
            "hostname": f"prod-analytics-01-sql.database.windows.net",
            "port":1433}    
    else:
        raise ValueError("Invalid environment: must be 'dev','qa', 'stg' or 'prod'.")


In [0]:
def build_jdbc_url(param_database,param_environment):
    v_jdbc_database = f"{param_database}"
    v_server_config         = get_server_config(param_environment)
    v_hostname              = v_server_config.get("hostname")
    v_port                  = v_server_config.get("port")
    v_jdbc_url = (
        f"jdbc:sqlserver://{v_hostname}:{v_port};"
        f"databaseName={v_jdbc_database};encrypt=true;trustServerCertificate=false;"
        f"hostNameInCertificate=*.database.windows.net;loginTimeout=30;"
    )

    return v_jdbc_url

In [0]:
def build_connection_properties(param_environment):
    v_server_config         = get_server_config(param_environment)
    v_username              = v_server_config.get("username")
    v_password              = v_server_config.get("password")
    v_connection_properties_dict = {
        "user"    : v_username,
        "password": v_password,
        "driver"  : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }
    return v_connection_properties_dict

In [0]:
def execute_dbconfig_sql_query(param_query,param_environment):
    v_formatted_query            = f"({param_query}) as tmp"
    v_database                   = f'db_Configuration'
    v_jdbc_url                   = build_jdbc_url(v_database, param_environment)
    v_connection_properties_dict = build_connection_properties(param_environment)              

    return spark.read.jdbc(
         url=v_jdbc_url,
         table=v_formatted_query,
         properties=v_connection_properties_dict)

In [0]:
def get_locations_by_env(param_environment):
    try:
        v_query              = f"SELECT AttributeName, AttributeValue FROM [CFG].[AttributeSystem] WHERE AttributeName IN ('AdlsAnalyticsFullpathUri','AdlsAnalyticsFullpathUriArchive','AdlsAnalyticsSecretScope','Environment','AnalyticsUnityCatalog')" 
                    
        v_system_attribute_df        = execute_dbconfig_sql_query(v_query, param_environment)
        v_system_attribute_dict      = {row["AttributeName"]: row["AttributeValue"] for row in v_system_attribute_df.collect()}
        v_locations_dict             ={
                                    'source'         : v_system_attribute_dict.get("AdlsAnalyticsFullpathUri"),
                                    'archive'        : v_system_attribute_dict.get("AdlsAnalyticsFullpathUriArchive"),
                                    'keyvault'       : v_system_attribute_dict.get("AdlsAnalyticsSecretScope"),
                                    'environment'    : param_environment,
                                    'unity_catalog'  : v_system_attribute_dict.get("AnalyticsUnityCatalog")
                                        }
    except Exception as e:
        raise Exception(f"Error: {e}")
    return v_locations_dict

In [0]:
def execute_dbconfig_stored_procedure(param_stored_procedure_string,param_environment):
    v_server_config         = get_server_config(param_environment)
    v_username              = v_server_config.get("username")
    v_password              = v_server_config.get("password")
    v_hostname              = v_server_config.get("hostname")
    v_jdbc_database         = "db_Configuration"
    v_query                 = param_stored_procedure_string
    try:
        with pymssql.connect(
            server=v_hostname,
            user=v_username,
            password=v_password,
            database=v_jdbc_database
        ) as v_conn:
            with v_conn.cursor() as v_cursor:
                v_cursor.execute(v_query)
                v_conn.commit()
    except Exception as e:
        raise Exception(f"Error: {e}")

In [0]:
def get_server_process_list(param_environment):
    v_query = (
        f"SELECT * FROM cfg.vw_DataFabricServerList "
    )
    v_query = f"({v_query})"
    v_formatted_query = v_query           
    v_server_process_list_df = execute_dbconfig_sql_query(v_formatted_query,param_environment)

    return v_server_process_list_df

In [0]:
def get_table_process_list(param_environment):
    v_query = (
        f"SELECT * FROM cfg.vw_DataFabricTableSplitList "
    )
    v_formatted_query = f"({v_query})"
    v_table_process_list_df = execute_dbconfig_sql_query(v_formatted_query,param_environment)
    return v_table_process_list_df

In [0]:
def update_process_status_facility_detail(param_status,
                                          param_InternalProductId,
                                          param_InternalClientId,
                                          param_InternalFacilityId,
                                          param_StepName,
                                          param_DataSourceId,
                                          param_environment,
                                          param_HWM = None):
    try:
        if param_status.upper() in ['P','F','S']:
            v_formatted_query = (
            f"SELECT * "
            f"FROM CFG.ProcessStatusFacilityDetail WHERE InternalProductId = '{param_InternalProductId}' "
                f"and InternalClientIds = '{param_InternalClientId}' "
                f"and InternalFacilityId = '{param_InternalFacilityId}' "
                f"and StepName = '{param_StepName}' "
                f"and DataSourceId = '{param_DataSourceId}' "
                )
            v_processStatusFacilityDetail_df = execute_dbconfig_sql_query(v_formatted_query,param_environment)
            if not v_processStatusFacilityDetail_df.isEmpty():
                
                if param_HWM == None:
                    execute_dbconfig_stored_procedure(
                    f"""
                    EXEC CFG.UpdateProcessStatusFacilityDetail @InternalProductId = {param_InternalProductId}, @InternalClientId = {param_InternalClientId}, 
                    @InternalFacilityId = {param_InternalFacilityId}, @StepName = '{param_StepName}', @Status = '{param_status}', @DataSourceId = {param_DataSourceId}
                    """
                    ,param_environment
                    )
                else:
                    execute_dbconfig_stored_procedure(
                    f"""
                    EXEC CFG.UpdateProcessStatusFacilityDetail @InternalProductId = {param_InternalProductId}, @InternalClientId = {param_InternalClientId}, 
                    @InternalFacilityId = {param_InternalFacilityId}, @StepName = '{param_StepName}', @Status = '{param_status}', @NewWatermarkValue = {param_HWM}, @DataSourceId = {param_DataSourceId}
                    """
                    ,param_environment
                    )
            else:
                print("Missing record please run the Implementation to insert records")
            
        else:
            raise ValueError("Invalid status - must be P for In Progress, S for Success or F for Failed")
    except Exception as e:
        raise Exception(f"Error: {e}")